# EE 582: Homework 8
### Eric D. Stevens
### March 5, 2019


In [ ]:
import numpy as np
import numpy.fft

## DFT Implementation

For the base case of the FFT I am using my submission form HW 7

In [ ]:
# dft snippet, exactly the same as submitted in HW 7
def dft(x, N=None, inverse=False):
    """DFT, ~6 lines"""
    L = len(x)
    if N is None:
        N = L

    # this is ugly, i know, sorry
    '''*************************************************************************'''
    X = np.array([((1/float(N))**(inverse))*np.sum(x[:N]*np.exp(((-1)**(not inverse))*2j*np.pi*k*np.arange(L)[:N]/N)) 
                  for k in np.arange(N)])
    '''*************************************************************************'''
    return X

### FFT Implementation

Here are the functions for the FFT and the IFFT.

In [3]:


# Radix-2 Fast Fourier Transform w/ inverse flag
def fft(x, N=None, inverse=False):
    
    """FFT implementation, ~14 lines"""
    L = len(x)
    if N is None:
        N = L 

    # in the case that len(x) > n
    if N<L:
        x = x[:N]
        L=N

    assert N % 2 == 0, 'N must be a power of 2'
    if N > 4:  # ad hoc threshold
        xn_evn = [x[i] for i in np.arange(L/2, dtype=int)*2]
        xn_odd = [x[i] for i in np.arange(L/2, dtype=int)*2+1]

        Xk_evn = fft(xn_evn, N=(N//2), inverse=inverse)# recursively calling ourselves
        out_exp = np.exp(np.arange((N/2))*((-1)**(not inverse))*2j*np.pi/N)
        Xk_odd = out_exp*fft(xn_odd, N=(N//2), inverse=inverse)# recursively calling ourselves

        Xk = np.zeros(N, dtype=np.complex128)
        for i in range(int(N/2)):
            Xk[i] = Xk_evn[i] + Xk_odd[i]
            Xk[i+int(N/2)] = Xk_evn[i] - Xk_odd[i]

        return ((1/2)**(inverse))*Xk

    else:
        return  dft(x, int(N), inverse=inverse)  # use either your solution, the official answer, or numpy's fft here


# Fast call to inverse Rad2 FFT
def ifft(X, N=None):
    return fft(X, N, inverse=True)



### Evaluation

I have made changes to the evaluation script because I believe that there was a bug that was causing improper testing of the IFFT function. Also added some explicit output to the script. The bug in the original script can be exposed by showing these outputs.

In [6]:
# Test script with bug fix and confirmation output
for N in [32,64,128]:
    
    x = np.random.randint(-128, 128, 64)
    X = fft(x, N)
    X_gold = np.fft.fft(x, N)
    
    assert np.allclose(X, X_gold)
    print('passed test fft: N = ', N, 'L = ', len(x))
    
    X = np.random.randint(-128, 128, 64)
    x = ifft(X, N)
    x_gold = np.fft.ifft(X, N)
    

    assert np.allclose(x, x_gold)
    print('passed test ifft: N = ', N, 'L = ', len(X))



passed test fft: N =  32 L =  64
passed test ifft: N =  32 L =  64
passed test fft: N =  64 L =  64
passed test ifft: N =  64 L =  64
passed test fft: N =  128 L =  64
passed test ifft: N =  128 L =  64


### Bug

You can see that whenever an IFFT is evaluated it is being done for N=L due to the usage of the same variable name that as the FFT.

In [7]:
for N in [32, 64, 128]:
    x = np.random.randint(-128, 128, 64)
    X = fft(x, N)
    X_gold = np.fft.fft(x, N)
    assert np.allclose(X, X_gold)
    
    # print statment for bug catching
    print('passed test fft: N = ', N, 'L = ', len(x))
    
    x = ifft(X, N)
    x_gold = np.fft.ifft(X, N)
    assert np.allclose(x, x_gold)
    print('passed test ifft: N = ', N, 'L = ', len(X))

passed test fft: N =  32 L =  64
passed test ifft: N =  32 L =  32
passed test fft: N =  64 L =  64
passed test ifft: N =  64 L =  64
passed test fft: N =  128 L =  64
passed test ifft: N =  128 L =  128
